===============
画图：积分型取平均的EI采集函数取点的演示图
===============
具体的实现是，导入30次BO数据点的json文件，提取最优的那一组超参数配置，固定除了lr之外的其他超参（保持与最优配置里的一样），相当于把6维的超参映射到lr这一维度，画出lr相关的采集函数图

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
import scipy.stats as stats
import os
import random
import time
import gc


tfd = tfp.distributions
tf.keras.backend.set_floatx('float64') # 确保数据类型一致

class BayesianOptimizer:
    def __init__(self, search_space, init_points=5, exploration=0.01, 
                 log_file="custom_bo_log.json", initial_X=None, initial_y=None):
        self.search_space = np.array(search_space)
        self.dim = len(search_space)
        self.init_points = init_points
        self.exploration = exploration
        self.log_file = log_file
        #----以防训练由于超时中断,设置自断点处接着BO迭代,加载已计算好的数据
        if initial_X is not None and initial_y is not None:
            self.X_obs = initial_X
            self.y_obs = initial_y
            print(f"BO optimizer has used {len(self.y_obs)} observed datas to initialize")
            #日志回填
            print(f"new log '{self.log_file}' is set and loads old data into it ")
            if os.path.exists(self.log_file):
                os.remove(self.log_file)
            for i in range(len(self.y_obs)):
                self._write_log(iteration=i + 1, x_new=self.X_obs[i], y_new=self.y_obs[i])
            print(f"历史数据回填完成。共写入 {len(self.y_obs)} 条记录。")
        else: 
            self.X_obs = np.zeros((0, self.dim))
            self.y_obs = np.zeros(0)
            if os.path.exists(self.log_file):
                os.remove(self.log_file)
                
        self.theta_dim = self.dim + 2 # length_scales (dim) + amplitude + noise
        
        self.num_samples = 20
        self.burn_in = 100
        
    
        
    def random_sample(self):
        """在搜索空间内随机采样一个超参数点。"""
        return np.array([np.random.uniform(low, high) for (low, high) in self.search_space])

    @tf.function(jit_compile=True) # 使用XLA编译加速
    def matern52_kernel(self, X1, X2, theta):
        X1_tf = tf.cast(X1, tf.float64)
        X2_tf = tf.cast(X2, tf.float64)
        theta = tf.cast(theta, tf.float64)
        
        length_scales = theta[:self.dim]
        amplitude = theta[self.dim]
        
        X1_scaled = X1_tf / length_scales
        X2_scaled = X2_tf / length_scales

        r2 = tf.reduce_sum(tf.square(X1_scaled), axis=1, keepdims=True) - 2 * tf.matmul(X1_scaled, X2_scaled, transpose_b=True) + tf.reduce_sum(tf.square(X2_scaled), axis=1)
        r2 = tf.maximum(r2, 1e-12)
        r = tf.sqrt(r2)
        
        five_64 = tf.constant(5.0, dtype=tf.float64)
        three_64 = tf.constant(3.0, dtype=tf.float64)
        one_64 = tf.constant(1.0, dtype=tf.float64)
        
        sqrt5_64 = tf.sqrt(five_64)
        
        term = (one_64 + sqrt5_64 * r + (five_64 / three_64) * r2) * tf.exp(-sqrt5_64 * r)
        
        K = amplitude**2 * term
        return K

    
    @tf.function(jit_compile=True) # 使用XLA编译加速
    def log_posterior(self, theta, X, y):
        f64 = tf.constant(1e-6, dtype=tf.float64)
        two_pi_64 = tf.constant(2.0 * np.pi, dtype=tf.float64)
        half_64 = tf.constant(0.5, dtype=tf.float64)
        
        X_tf = tf.cast(X, dtype=tf.float64)
        y_tf = tf.cast(y, dtype=tf.float64)
        theta_tf = tf.cast(theta, dtype=tf.float64)
        
        noise = theta_tf[self.dim + 1]**2
        
        K = self.matern52_kernel(X_tf, X_tf, theta_tf)
        n = tf.shape(K)[0]
        K += (noise + f64) * tf.eye(n, dtype=tf.float64)
        
        L = tf.linalg.cholesky(K)
        alpha = tf.linalg.cholesky_solve(L, tf.expand_dims(y_tf, 1))
        
        log_lik = -half_64 * tf.squeeze(tf.matmul(tf.expand_dims(y_tf, 0), alpha))
        log_lik -= tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)))
        n_64 = tf.cast(n, tf.float64)
        log_lik -= half_64 * n_64 * tf.math.log(two_pi_64)
        
        return log_lik


    def sample_theta_posterior(self):
        def target_log_prob_fn(theta):
            return self.log_posterior(theta, self.X_obs, self.y_obs)

        kernel = tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=target_log_prob_fn,
            num_leapfrog_steps=3,
            step_size=0.1)
        
        samples = tfp.mcmc.sample_chain(
            num_results=self.num_samples,
            current_state=tf.ones(self.theta_dim, dtype=tf.float64),
            kernel=kernel,
            num_burnin_steps=self.burn_in,
            trace_fn=None)
        
        return samples.numpy()
    
    def expected_improvement(self, x_candidate, theta_samples):
        if self.y_obs.size == 0: return 1.0
        
        y_best = np.max(self.y_obs)
        x_candidate_reshaped = x_candidate.reshape(1, -1)
        
        ei_values = []
        for theta in theta_samples:

            try:
                theta_np = theta.numpy() if hasattr(theta, 'numpy') else theta
                
                K_obs = self.matern52_kernel(self.X_obs, self.X_obs, theta_np).numpy()
                noise = theta_np[self.dim + 1]**2
                K_obs += (noise + 1e-6) * np.eye(len(self.X_obs))
                
                K_cross = self.matern52_kernel(self.X_obs, x_candidate_reshaped, theta_np).numpy()
                K_candidate = self.matern52_kernel(x_candidate_reshaped, x_candidate_reshaped, theta_np).numpy()
                L = np.linalg.cholesky(K_obs)
                L_inv_K_cross = np.linalg.solve(L, K_cross)
                
                mu = np.dot(L_inv_K_cross.T, np.linalg.solve(L, self.y_obs))
                sigma2 = K_candidate - np.dot(L_inv_K_cross.T, L_inv_K_cross)

                mu_scalar = mu[0]
                sigma2_scalar = sigma2[0, 0]
                
                sigma = np.sqrt(np.maximum(sigma2_scalar, 1e-9))
                if sigma == 0.0:
                    continue

                z = (mu_scalar - y_best - self.exploration) / sigma
                
                norm = stats.norm(loc=0, scale=1)
                ei = (mu_scalar - y_best - self.exploration) * norm.cdf(z) + sigma * norm.pdf(z)
                
                if not np.isnan(ei):
                    ei_values.append(ei)

            except np.linalg.LinAlgError:
                print("警告: 在EI计算中发生矩阵奇异值错误，跳过此theta样本。")
                continue
                
        return np.mean(ei_values) if ei_values else 0.0
         

    
    def _write_log(self, iteration, x_new, y_new):
        param_names = ['lr','dense_units','dropout_rate','l2_reg','optimizer_choice','momentum']
        params_dict = {name:val for name, val in zip(param_names, x_new)}
        log_entry={'iteration':int(iteration),
                   'accuracy':float(y_new),
                   'params':params_dict}
        with open(self.log_file,'a') as f: 
            f.write(json.dumps(log_entry)+'\n')
    
    def optimize(self, objective_fn, n_iter=30):
        #检查是 初次BO 还是 接续BO
        num_existing_points=self.X_obs.shape[0]
        num_init_points_to_run = max(0, self.init_points - num_existing_points)

        if num_init_points_to_run > 0:
            # 假如加载的点数少于要求的初始随机化点数，需要补充运行
            print(f"--- 继续随机初始化阶段: 还需要运行 {num_init_points_to_run} 个初始点。 ---")
            for i in range(num_init_points_to_run):
                print(f"\n--- 初始点 {num_existing_points + i + 1}/{self.init_points} ---")
                x_new = self.random_sample()
                y_new = objective_fn(x_new)
                self.X_obs = np.vstack([self.X_obs, x_new])
                self.y_obs = np.append(self.y_obs, y_new)
                self._write_log(iteration=num_existing_points + i + 1, x_new=x_new, y_new=y_new)
        
        elif num_existing_points > 0:
            # 成功恢复运行。加载的点数已满足要求，跳过随机搜索。
            print(f"--- 成功恢复优化: 已加载 {num_existing_points} 个点。跳过 {self.init_points} 个初始点的随机搜索阶段。 ---")
        
        else:
            #全新运行 (没有加载任何数据)
            print(f"--- 开始全新运行: 执行 {self.init_points} 个初始随机点 ---")
            for i in range(self.init_points):
                print(f"\n--- 初始点 {i + 1}/{self.init_points} ---")
                x_new = self.random_sample()
                y_new = objective_fn(x_new)
                self.X_obs = np.vstack([self.X_obs, x_new])
                self.y_obs = np.append(self.y_obs, y_new)
                self._write_log(iteration=i + 1, x_new=x_new, y_new=y_new)

        print(f"\n--- 开始执行 {n_iter} 次新的贝叶斯优化迭代 ---")

        
        for i in range(n_iter):
            print(f"\n--- 优化迭代 {i+1}/{n_iter} ---")
            
            theta_samples = self.sample_theta_posterior()
            
            best_ei = -1
            best_x = None
            for _ in range(200): # 增加随机搜索次数以更好地优化采集函数
                x_candidate = self.random_sample()
                ei = self.expected_improvement(x_candidate, theta_samples)
                if ei > best_ei:
                    best_ei = ei
                    best_x = x_candidate
            
            if best_x is None:
                print("警告：无法找到有效的候选点，将进行随机采样。")
                best_x = self.random_sample()

            y_new = objective_fn(best_x)
            
            self.X_obs = np.vstack([self.X_obs, best_x])
            self.y_obs = np.append(self.y_obs, y_new)
            # 在每次优化后也写入日志 ---
            new_iteration_number = self.X_obs.shape[0]
            self._write_log(iteration=new_iteration_number, x_new=best_x, y_new=y_new)


        best_idx = np.argmax(self.y_obs)
        return self.X_obs[best_idx], self.y_obs[best_idx]


    def get_slice_visualization_data(self, x_anchor, var_param_index, num_grid_points=100):
        #x_anchor: 6维“锚点”，即BO找到的当前最佳点
        #var_param_index:切片的投影维度
        #num_grid_points:横坐标网格数量,用来绘制平滑曲线
        print("正在使用HMC采样GP的超参数 (theta)...")
        # 1. 获取 theta 样本
        try:
            theta_samples = self.sample_theta_posterior() 
        except tf.errors.InvalidArgumentError as e:
            print(f"错误：HMC采样失败。{e}")
            return None
            
        # 2. 定义 1D 切片 (X-axis)
        var_space = self.search_space[var_param_index]
        is_log_scale = (var_space[1] / var_space[0] > 100) 
        if is_log_scale:
            x_axis = np.logspace(np.log10(var_space[0]), np.log10(var_space[1]), num_grid_points)
        else:
            x_axis = np.linspace(var_space[0], var_space[1], num_grid_points)

        # 3. 创建 6D 测试网格
        X_grid = np.tile(x_anchor, (num_grid_points, 1))
        #np.tile就是行复制
        X_grid[:, var_param_index] = x_axis

        # 4. 循环遍历每个 theta 样本
        y_best = np.max(self.y_obs)
        all_mu_curves = []
        all_sigma_curves = []
        all_ei_curves = []
        
        print(f"正在为 {len(theta_samples)} 个 theta 样本计算后验和EI曲线...")
        for theta in theta_samples:
            try:
                # ... (GP回归和EI计算) ...
                theta_np = theta.numpy() if hasattr(theta, 'numpy') else theta
                noise = theta_np[self.dim + 1]**2 + 1e-6
                K_obs = self.matern52_kernel(self.X_obs, self.X_obs, theta_np).numpy()
                K_obs += np.eye(len(self.X_obs)) * noise
                K_cross = self.matern52_kernel(self.X_obs, X_grid, theta_np).numpy()
                K_grid = self.matern52_kernel(X_grid, X_grid, theta_np).numpy()
                L = np.linalg.cholesky(K_obs)
                alpha_ = np.linalg.solve(L, self.y_obs)
                alpha = np.linalg.solve(L.T, alpha_)
                mu_grid = K_cross.T @ alpha
                v = np.linalg.solve(L, K_cross) 
                var_grid = np.diag(K_grid) - np.sum(v**2, axis=0)
                sigma_grid = np.sqrt(np.maximum(var_grid, 1e-9))
                z = (mu_grid - y_best - self.exploration) / sigma_grid
                norm_dist = stats.norm(loc=0, scale=1)
                ei_grid = (mu_grid - y_best - self.exploration) * norm_dist.cdf(z) + sigma_grid * norm_dist.pdf(z)
                ei_grid[sigma_grid == 0.0] = 0.0
                all_mu_curves.append(mu_grid)
                all_sigma_curves.append(sigma_grid)
                all_ei_curves.append(ei_grid)
            except np.linalg.LinAlgError:
                continue 

        # 5. 计算积分EI
        integrated_ei = np.mean(all_ei_curves, axis=0)

        print("✅ 切片数据计算完成。")
        return x_axis, all_mu_curves, all_sigma_curves, all_ei_curves, integrated_ei, is_log_scale

   
            


In [ ]:
# ===================================================================
#加载数据并实例化优化器
# ===================================================================

search_space = [
    (1e-5, 5e-4),    # lr
    (64, 512),       # dense_units
    (0.2, 0.7),      # dropout_rate
    (1e-5, 5e-3),    # l2_reg
    (0, 3.99),      # 0=Adam, 1=SGD, 2=RMSprop, 3=Adagrad
    (0.85, 0.99)     # momentum
]



#加载日志文件-继续上次运行
log_file_path = "/kaggle/input/bo-data-30/second_continued_EI_MCMC_BO_log30.json" 

param_order=['lr', 'dense_units', 'dropout_rate', 'l2_reg', 'optimizer_choice', 'momentum']
X_list_full = []
y_list_full = []
num_points_to_load = 10 
print(f"--- 正在从 '{log_file_path}' 提取前 {num_points_to_load} 条数据 ---")
try:
    with open(log_file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= num_points_to_load: 
                break
            
            line = line.strip()
            if not line: continue
            
            log_entry = json.loads(line)
            y_list_full.append(log_entry['accuracy'])
            params_dict = log_entry['params']
            X_list_full.append([params_dict[key] for key in param_order])

    initial_X_early = np.array(X_list_full)
    initial_y_early = np.array(y_list_full)

    if len(initial_y_early) < num_points_to_load:
        print(f"❌ 警告: 只加载了 {len(initial_y_early)} 个点，少于目标的 {num_points_to_load} 个。")

    print(f"成功加载了 {len(initial_y_early)} 条历史数据。")

    optimizer_early_stage = BayesianOptimizer(
        search_space=search_space,
        log_file="visualization_log_early.json", 
        initial_X=initial_X_early,
        initial_y=initial_y_early
    )
    print(" 早期阶段优化器实例已创建，并加载了 5 个数据点。")
    print(f"   - 内部 X_obs 形状: {optimizer_early_stage.X_obs.shape}")
    print(f"   - 内部 y_obs 形状: {optimizer_early_stage.y_obs.shape}")

except FileNotFoundError:
    print(f"❌ 错误: 日志文件未找到，请检查路径: '{log_file_path}'")
    optimizer_early_stage = None
except Exception as e:
    print(f"❌ 加载数据时出错: {e}")
    optimizer_early_stage = None
    



In [ ]:
# ===================================================================
# 运行可视化绘图 (标签版)
# ===================================================================

from matplotlib import rcParams 
rcParams['font.weight'] = 'bold'
rcParams['axes.labelweight'] = 'bold'

# 检查“早期阶段”优化器对象是否存在 ---
if 'optimizer_early_stage' in locals() and optimizer_early_stage.X_obs.shape[0] > 0:

    current_best_idx = np.argmax(optimizer_early_stage.y_obs)
    x_anchor = optimizer_early_stage.X_obs[current_best_idx]
    var_param_index = 1 
    var_param_name = 'dense_units' 
    vis_data = optimizer_early_stage.get_slice_visualization_data(
        x_anchor=x_anchor,
        var_param_index=var_param_index,
        num_grid_points=200 
    )
else:
    print("❌ 错误: 'optimizer_early_stage' 对象未准备好。请先运行上一个单元格。")
    vis_data = None

# 开始绘图 (3个子图) ---
if vis_data:
    x_axis, all_mu, all_sigma, all_ei, integrated_ei_all, is_log_scale = vis_data
    
    print("正在绘制图表...")
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 7), sharex=True)
    
    latex_labels = [r'Posterior Mean $\mu(\mathbf{h} ; \psi^{(1)})$', 
                    r'Posterior Mean $\mu(\mathbf{h} ; \psi^{(2)})$', 
                    r'Posterior Mean $\mu(\mathbf{h} ; \psi^{(3)})$']
    ei_labels = [r'$a_{\mathrm{EI}}(\mathbf{h} ; \psi^{(1)}$)', 
                 r'$a_{\mathrm{EI}}(\mathbf{h} ; \psi^{(2)}$)', 
                 r'$a_{\mathrm{EI}}(\mathbf{h} ; \psi^{(3)}$)']
    
    legend_props = {'loc': 'lower right', 'prop': {'weight': 'bold', 'size': 14}}

    # --- (a) 绘制后验样本 (Posterior Samples) ---
    colors = ['C3', 'C2', 'C0'] # (红, 绿, 蓝)
    for i in range(min(3, len(all_mu))):
        mu, sigma = all_mu[i], all_sigma[i]
        axes[0].plot(x_axis, mu, color=colors[i], lw=2, label=latex_labels[i])
        axes[0].fill_between(x_axis, mu - 1.96 * sigma, mu + 1.96 * sigma, 
                             color=colors[i], alpha=0.15) # 重新启用置信区间
    
    # 将标题放回顶部
    axes[0].set_title('(a) Posterior Predictive Means', fontsize=20, fontweight='bold')
    axes[0].set_xlabel('') # 顶部图表不需要 X 标签
    axes[0].set_ylabel('Expected Validation Accuracy', fontsize=18, fontweight='bold')
    axes[0].legend(**legend_props) 

    # --- (b) 绘制对应的 EI 曲线 ---
    ei_curves_to_plot = [] 
    for i in range(min(3, len(all_ei))):
        ei = all_ei[i]
        ei_curves_to_plot.append(ei) 
        axes[1].plot(x_axis, ei, color=colors[i], lw=2, label=ei_labels[i])
        max_ei_idx = np.argmax(ei)
        axes[1].plot(x_axis[max_ei_idx], ei[max_ei_idx], marker='o', 
                     color=colors[i], markersize=12, markeredgecolor='black')
    
    axes[1].set_title('(b) Expected Improvement Functions', fontsize=20, fontweight='bold')
    axes[1].set_xlabel('') # 中间图表不需要 X 标签
    axes[1].set_ylabel('Expected Improvement (EI)', fontsize=18, fontweight='bold')
    axes[1].legend(**legend_props) 

    # --- (c) 绘制 【前三条EI曲线】 的平均值 ---
    if ei_curves_to_plot:
        integrated_ei_3 = np.mean(ei_curves_to_plot, axis=0) 
        
        axes[2].plot(x_axis, integrated_ei_3, color='black', lw=2.5, 
                     label=r'Integrated EI $a_{\mathrm{BO}}(\mathbf{h})$')
        
        max_iei_idx = np.argmax(integrated_ei_3)
        axes[2].plot(x_axis[max_iei_idx], integrated_ei_3[max_iei_idx], marker='o', 
                     color='black', markersize=12, label='Chosen Next Point')
    
    axes[2].set_title('(c) The Integrated Acquisition Function', fontsize=20, fontweight='bold')
    axes[2].set_ylabel('Integrated EI', fontsize=18, fontweight='bold')
    axes[2].legend(**legend_props) 
    
    if is_log_scale: axes[2].set_xscale('log') 
    
    for ax in axes:
        ax.grid(True, linestyle='--', alpha=0.6)
        if is_log_scale: ax.set_xscale('log')
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')
            
    plt.tight_layout() 
    
    plt.savefig('full_bo_visualization_slice_early_stage.pdf', format='pdf', bbox_inches='tight')
    plt.show()

plt.rcdefaults()
print("\n[INFO] Matplotlib 字体设置已重置为默认值。")